# [**Facial Keypoints Detection Project**](https://www.kaggle.com/c/facial-keypoints-detection)

# Deep Learning Course, IASBS

Alireza Mansouri in partnership with Mahdi Nasrollahi, *Supervisor : Dr Parvin Razzaghi*

# List Input Files

In [ ]:
import os
os.listdir("../input/facial-keypoints-detection")

# Importing the required libraries

In [ ]:
import pandas as pd
import numpy as np
import keras
from tqdm import tqdm
from keras import backend as K
from matplotlib import pyplot as plt

# Reading the Data from csv files and convert them to dataframe

In [ ]:
lookid_data = pd.read_csv("../input/facial-keypoints-detection/IdLookupTable.csv")
lookid_data.head()

In [ ]:
samplesubmission = pd.read_csv("../input/facial-keypoints-detection/SampleSubmission.csv")
samplesubmission.head()

In [ ]:
train = pd.read_csv("../input/facial-keypoints-detection/training.zip")
train.head().T

# Data Cleaning

isnull(). sum() will give the column-wise sum of missing values. This returns the counts of non-NA, NA and total number of entries per group.

In [ ]:
train.isnull().sum()

method='ffill': Ffill or forward-fill propagates the last observed non-null value forward until another non-null value is encountered.

In [ ]:
#filling the nan values
train.fillna(method = 'ffill',inplace = True)

# **Preparing the training data**

In [ ]:
X = train.Image.values
del train['Image']
Y = train.values

# Converting string images data to numeric values.

In [ ]:
x = []
for i in tqdm(X):
    q = [int(j) for j in i.split()]
    x.append(q)
len(x)

In [ ]:
x = np.array(x)
x = x.reshape(7049, 96,96,1)
x  = x/255.0
x.shape

# Splitting the data into 90-10 train test split

In [ ]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test = tts(x,Y,random_state = 69,test_size = 0.1)

In [ ]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

# ** Model Implementation **

In [ ]:
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
model.compile(optimizer = 'adam',loss = 'mean_squared_error', metrics = ['mae','acc'])
model.fit(x_train,y_train,batch_size=256, epochs=50,validation_data=(x_test,y_test))

# Training on the complete Dataset now

In [ ]:
model.compile(optimizer = 'adam',loss = 'mean_squared_error', metrics = ['mae'])
model.fit(x,Y,batch_size=64, epochs=100)
model.fit(x,Y,batch_size=128, epochs=50)
model.fit(x,Y,batch_size=256, epochs=50)

# Predicting for test data

In [ ]:
test = pd.read_csv("../input/facial-keypoints-detection/test.zip")
test.head()

In [ ]:
test.isnull().sum()

In [ ]:
test = test.Image.values
x_t = []
for i in tqdm(test):
    q = [int(j) for j in i.split()]
    x_t.append(q)
x_t = np.array(x_t)
x_t = x_t.reshape(-1, 96,96,1)
x_t = x_t/255.0
x_t.shape

In [ ]:
pred = model.predict(x_t)
pred.shape

# Select a number between 1 to 1782 for i.e

In [ ]:
n = 500

xv = x_train[n].reshape((96,96))
plt.imshow(xv,cmap='gray')

for i in range(1,31,2):
    plt.plot(pred[n][i-1], pred[n][i], 'ro')
    plt.plot(y_train[n][i-1], y_train[n][i], 'x', color='green')

plt.show()

n = 501

xv = x_train[n].reshape((96,96))
plt.imshow(xv,cmap='gray')

for i in range(1,31,2):
    plt.plot(pred[n][i-1], pred[n][i], 'ro')
    plt.plot(y_train[n][i-1], y_train[n][i], 'x', color='green')

plt.show()

n = 503

xv = x_train[n].reshape((96,96))
plt.imshow(xv,cmap='gray')

for i in range(1,31,2):
    plt.plot(pred[n][i-1], pred[n][i], 'ro')
    plt.plot(y_train[n][i-1], y_train[n][i], 'x', color='green')

plt.show()

In [ ]:
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)

In [ ]:
rowid = lookid_data['RowId']
rowid=list(rowid)

In [ ]:
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))

In [ ]:
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])

In [ ]:
rowid = pd.Series(rowid,name = 'RowId')

In [ ]:
loc = pd.Series(preded,name = 'Location')

In [ ]:
submission = pd.concat([rowid,loc],axis = 1)

# Exporting the Keypoints file

In [ ]:
submission.to_csv('Keypoints.csv',index = False)